# Process ACCESS-ESM1-5 to NEM region

In [3]:
from dask.distributed import Client,LocalCluster
from dask_jobqueue import PBSCluster

In [77]:
# One node on Gadi has 48 cores - try and use up a full node before going to multiple nodes (jobs)

walltime = '00:30:00'
cores = 48
memory = str(4 * cores) + 'GB'

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory), processes=cores,
                     job_extra_directives=['-q normal',
                                           '-P w42',
                                           '-l ncpus='+str(cores),
                                           '-l mem='+str(memory),
                                           '-l storage=gdata/w42+gdata/rt52+gdata/fs38'],
                     local_directory='$TMPDIR',
                     job_directives_skip=["select"])
                     # python=os.environ["DASK_PYTHON"])

In [78]:
cluster.scale(jobs=1)
client = Client(cluster)

In [88]:
client#.restart()

<Client: 'tcp://10.6.121.4:38771' processes=48 threads=48, memory=179.04 GiB>

In [7]:
import xarray as xr
import functools

import os

In [9]:
models = [
    "ACCESS-ESM1-5"
]

In [10]:
def get_model_path(model):
    """
    Return the path to the historical simulations
    """
    if model == "ACCESS-ESM1-5":
        return "/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/"
    else:
        raise ValueError("Incorrect model")

In [33]:
def get_model_grid(model):
    """
    Return the string for the model grid
    """
    if model == "ACCESS-ESM1-5":
        return "gn"
    else:
        raise ValueError("Incorrect model")

In [41]:
def sel_Aus(ds, lat_name="lat", lon_name="lon"):
    """
    Select Australian region.
    """
    return ds.sel({lon_name: slice(110, 155), lat_name: slice(-45, -10)})

In [45]:
def open_model_data(model, variable, time, preprocess):
    """
    Concat historical runs from 1950 for all ensemble members.
    
    model: str, model name.
    variable: str, variable name
    time: str, name of time interval e.g. day
    preprocess: function to pass to preprocess in open_mfdataset
    """
    model_path = get_model_path(model)
    model_grid = get_model_grid(model)
    members = os.listdir(model_path)
    
    ds_list = []
    for member in members:
        filename = variable + "_" + time + "_" + model + "_historical_" + member + "_" + model_grid
        path = model_path + member + "/" + time + "/" + variable + "/" + model_grid + "/latest/" + filename

        # Open data and preprocess
        ds = xr.open_mfdataset([
            path + "_19500101-19991231.nc",
            path + "_20000101-20141231.nc"
        ],
            join="exact",
            preprocess=preprocess,
            engine="netcdf4",
            compat="override",
            coords="minimal",
        )
        
        # Assign member dimension
        m = int(member.split("i")[0][1:])
        ds = ds.expand_dims({"member": [m]})
        ds_list.append(ds)
        
    return xr.concat(ds_list, dim="member").sortby("member")

In [47]:
def chunk_member(ds, chunks):
    """
    Re-chunk member dimension.
    """
    return ds.chunk({"member": chunks})

In [48]:
def to_zarr(ds, model, variable, time, region):
    """
    Write to zarr collection.
    """
    filename = variable + "_" + time + "_" + model + "_historical_1950-2014_" + region + ".zarr"
    path = "/g/data/w42/dr6273/work/data/CMIP6/" + model + "/historical/" + time + "/" + variable + "/" + filename
    ds.to_zarr(
        path,
        mode="w",
        consolidated=True
    )

In [80]:
def open_chunk_write(model, variable, time, region, preprocess, member_chunks=5):
    """
    Open model data, concat alomng member, chunk and write to zarr
    
    model: str, model name.
    variable: str, variable name
    time: str, name of time interval e.g. day
    region: str, name of region for filename
    preprocess: function to pass to preprocess in open_mfdataset
    member_chunks: how many chunks for members before writing
    """
    ds = open_model_data(model, variable, time, preprocess)
    ds = chunk_member(ds, member_chunks)
    to_zarr(ds, model, variable, time, region)

In [81]:
def preprocess(variable):
    """
    Preprocess for open_mfdataset. Uses functools.partial to return a function 
    that only takes one argument.
    """
    def _preprocess(d, v):
        if v in ["tas", "tasmax"]:
            d = d.drop("height")
        d = d[v].to_dataset(name=v)
        return sel_Aus(d)
    
    return functools.partial(_preprocess, v=variable)

### Temperature (2m)

In [74]:
# def preprocess_tas(ds):
#     ds = ds["tas"].to_dataset(name="tas")
#     ds = ds.drop("height")
#     return sel_Aus(ds)

In [75]:
%%time
open_chunk_write(
    "ACCESS-ESM1-5",
    "tas",
    "day",
    "AUS",
    preprocess("tas")
)

CPU times: user 14.2 s, sys: 2.08 s, total: 16.3 s
Wall time: 59.2 s


### Daily max temperature (2m)

In [89]:
# def preprocess_tasmax(ds):
#     ds = ds["tasmax"].to_dataset(name="tasmax")
#     ds = ds.drop("height")
#     return ds.sel(lon=slice(133, 155), lat=slice(-45, -10))

In [90]:
%%time
open_chunk_write(
    "ACCESS-ESM1-5",
    "tasmax",
    "day",
    "AUS",
    preprocess("tasmax")
)

CPU times: user 1min 10s, sys: 16.9 s, total: 1min 27s
Wall time: 6min 35s


### Shortwave radiation at surface

In [91]:
# def preprocess_rsds(ds):
#     ds = ds["rsds"].to_dataset(name="rsds")
#     return ds.sel(lon=slice(133, 155), lat=slice(-45, -10))

In [ ]:
%%time
open_chunk_write(
    "ACCESS-ESM1-5",
    "rsds",
    "day",
    "AUS",
    preprocess("rsds")
)

### Surface wind speed (~10 m)

In [ ]:
# def preprocess_sfcWind(ds):
#     ds = ds["sfcWind"].to_dataset(name="sfcWind")
#     return ds.sel(lon=slice(133, 155), lat=slice(-45, -10))

In [ ]:
%%time
open_chunk_write(
    "ACCESS-ESM1-5",
    "sfcWind",
    "day",
    "AUS",
    preprocess("sfcWind")
)

### Precip

In [ ]:
# def preprocess_pr(ds):
#     ds = ds["pr"].to_dataset(name="pr")
#     return ds.sel(lon=slice(133, 155), lat=slice(-45, -10))

In [ ]:
%%time
open_chunk_write(
    "ACCESS-ESM1-5",
    "pr",
    "day",
    "AUS",
    preprocess("pr")
)

# Close cluster

In [16]:
client.close()
cluster.close()